In [1]:
###### -*- coding: utf-8 -*-
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from pandas import Series, DataFrame
import matplotlib.cm as cm
from math import *
import collections
import seaborn as sns
import time
import random
import itertools
import networkx as nx
import collections
mylen = np.vectorize(len)

In [2]:
######Initial state: random sequences.
def initial6(L, N, Char, X): # X number of randomly selected identical sequences.
    population = [''.join(np.random.choice(Char, L, replace=True)) for i in range(X)]
    population = population*int(N/X)
    np.random.shuffle(population) 
    return population

#####Analyze closed networks
def time_limited_cycles(G, time_limit=2.0):
    elapsed, cycle_generator = 0.0, nx.simple_cycles(G)
    while elapsed <= time_limit:
        start_time = time.time()
        try:
            cycle = next(cycle_generator)
        except StopIteration:
            break
        yield cycle
        elapsed += time.time() - start_time 

##### Sample all the indexes when a particular string matches in another string.
def find_all(st, substr, start_pos=0, accum=[]):
    ix = st.find(substr, start_pos)
    if ix == -1: #no match
        return accum
    return find_all(st, substr, start_pos=ix + 1, accum=accum + [ix])

In [3]:
#####Template-directed ligation (very similar to the other program for Reproduction)
#####If S nt of 3' end of a substrate + S nt of 5' end of another substrate 
#form 6 consecutive bps with a template, they ligate.
def ligation(S):
    global ligation_num 
    idx = random.sample(range(len(population)),3) 
    template = population[idx[0]]
    monomer1 = population[idx[1]]
    monomer2 = population[idx[2]]
    rev_template = template[::-1] 
    rev_template = rev_template.translate(str.maketrans('AUGC', 'UACG')) 

    ##Ligation: A + B + template -> AB + template
    #assuming that monomer1 is B and monomer 2 is A
    if len(monomer1)>=S and len(monomer2)>=S:
        comp1_set = find_all(rev_template, monomer1[:S])           
        if len(comp1_set) >= 1:
            comp1 = random.choice(comp1_set)
            if comp1 >= S and rev_template[comp1-S:comp1] == monomer2[-S:]: 
                #comp2 == monomer2[-S:]
                ##ligation: 
                new_monomer1 = monomer2 + monomer1   

                #replace the original monomers (template is not changed)
                #add a molecule from initial population to fill in a ligated moleclue
                population[idx[1]] = new_monomer1 
                population[idx[2]] = np.random.choice(initial_population)
                ligation_num +=1
                catalysts.append(template)
                products.append(new_monomer1)
                monomer1s.append(monomer1)
                monomer2s.append(monomer2)

        else:#assuming that monomer 1 is AC and monomer 2 is B
            comp3_set = find_all(rev_template, monomer2[:S])
            if len(comp3_set) >= 1:
                comp3 = random.choice(comp3_set)                   
                if comp3 >= S and rev_template[comp3-S:comp3] == monomer1[-S:]:
                    #comp2 == monomer1[-S:]
                    new_monomer1 = monomer1 + monomer2
                    population[idx[1]] = new_monomer1
                    population[idx[2]] = np.random.choice(initial_population)
                    ligation_num +=1
                    catalysts.append(template)
                    products.append(new_monomer1)
                    monomer1s.append(monomer1)
                    monomer2s.append(monomer2)


#####calculate complexity of a graph
def complexity(thr, catalysts, products, ligation_num, final_population):
    G = nx.DiGraph()
    G.add_nodes_from(catalysts)
    G.add_nodes_from(products)
    edges = [(catalysts[i],products[i]) for i in range(ligation_num)]
    self_reproducers = [node0 for node0, node1 in edges if node0==node1]
    
    #weight: how many times each catalytic reaction occurred.
    for node0,node1 in edges:
        if G.has_edge(node0, node1):
                G[node0][node1]["weight"] += 1
        else:
            G.add_edge(node0, node1, weight = 1)

    #remove edges depending on "weight" threashold
    for (u,v,d) in list(G.edges(data=True)): #need 'list' otherwise, error appears (dictionary size changes)
        if d['weight'] < thr:
            G.remove_edge(u, v)

    #remove nodes that dose not exist in the final population
    node_set = list(G.nodes())
    non_exist_nodes = list(set(node_set) - set(final_population))
    G.remove_nodes_from(non_exist_nodes)

    #explore closed cycles (or self-reproduction) (permit overlap at this point)
    global cycle_num
    cycle_edges = []
    closed_cycles_or_self = list(time_limited_cycles(G)) #only above threshold, including self-reproduction
    cycle_num = len(closed_cycles_or_self)
    cycle_size = []
    for i in range(len(closed_cycles_or_self)):
        a = [(closed_cycles_or_self[i][j], closed_cycles_or_self[i][j+1])\
                             for j in range(len(closed_cycles_or_self[i])-1)]
        a = a + [(closed_cycles_or_self[i][-1], closed_cycles_or_self[i][0])]
        for k in range(len(a)):
            cycle_size.append(len(a))
        cycle_edges += a
    
    #calclate maximum cycle size
    global maxsize
    if len(cycle_size)>0:
        maxsize = max(cycle_size) 

    #Acquire information of weight for each edge
    cycle_weights = [G[cycle_edges[i][0]][cycle_edges[i][1]]['weight'] for i in range(len(cycle_edges))]

    #Make new graph by eliminating overlap of edges(to make each highest order remain)
    newG = nx.DiGraph()
    for i in range(len(cycle_edges)):
        node0, node1 = cycle_edges[i][0], cycle_edges[i][1]
        if newG.has_edge(node0, node1) and newG[node0][node1]["cycle_size"] < cycle_size[i]:
            newG.add_edge(node0,node1, cycle_size = cycle_size[i], weight= cycle_weights[i])
        elif newG.has_edge(node0, node1):       
            pass
        else:
            newG.add_edge(node0,node1, cycle_size = cycle_size[i], weight= cycle_weights[i])

    #calculate fraction of networks
    global fraction
    if len(edges)>0:
        new_weights = [newG[list(newG.edges())[i][0]][list(newG.edges())[i][1]]['weight'] \
                    for i in range(len(newG.edges()))]
        fraction = sum(new_weights)/len(edges)

    G.clear()

In [6]:
#####Initial condition
L = 8 # length of initial population (completely random)
S = 3 # requied complementary consecutive nucletides (for each side) for ligation
Char =['A', 'U','G','C']

#####Run
Xset = [2,5,10,20,40,80,160]
thr = 2
for l in range(len(Xset)):
    ligation_num_set, reaction_variation_set,  maxsize_set, fraction_set = [],[],[],[]
    X = Xset[l]
    N = 300*X #population size
    genmax = 100000*X #reaction step
    for k in range(1):
        gen=0
        ligation_num, maxsize, fraction = 0,0,0
        gen_set = []
        catalysts = []
        products = [] #ligated
        monomer1s, monomer2s = [],[]
        
        population = initial6(L, N, Char, X)
        initial_population = list(set(population))
        print(initial_population[:5])

        start = time.perf_counter()
        for i in range(genmax):
            ligation(S)
        final_population = population
        ligation_num_set.append(ligation_num)
        #calculate reaction variation
        reactions = [[catalysts[i],products[i]] for i in range(len(catalysts))]
        reactions_unique = list(map(list, set(map(tuple, reactions)))) #remove duplicated elements
        reaction_variation = len(reactions_unique)
        reaction_variation_set.append(reaction_variation)
        end = time.perf_counter()
        print('run time', end-start, 'sec')

        #####Calculation of complexity
        start = time.perf_counter()
        complexity(thr, catalysts, products, ligation_num, final_population)
        maxsize_set.append(maxsize)
        fraction_set.append(fraction)
        end = time.perf_counter()
        print('calculation time', end-start, 'sec')
        print('##########')

    
    ######DataFrame
    df = DataFrame({'ligation_num' : ligation_num_set,'reaction_variation' :reaction_variation_set,\
                        'maxsize':maxsize_set,'fraction':fraction_set}, \
                        columns = ['ligation_num','reaction_variation','maxsize','fraction'])
    df.to_csv('Ligation_300X_X='+str(X)+'_S='+str(S)+'_T='+str(thr)+'_gen='+str(genmax)+'.csv')
    print('Done')
df

['AAUGUUCU', 'CUAAGCGC', 'GGAUUGAU', 'UAAGUGAU', 'GGCUUACG']
run time 27.513604751000003 sec
calculation time 0.2542038219999938 sec
##########
Done


,ligation_num,reaction_variation,maxsize,fraction
0,3616,1054,23,0.136062
